In [1]:
print("test")

test


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

if os.environ['OPENAI_API_KEY']:
    print("API Key is set.")

API Key is set.


In [3]:
from langchain_openai import ChatOpenAI


In [4]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

### RAG implementation with pdf

### Extracting text from pdf

In [6]:
from langchain_community.document_loaders import PyPDFLoader

In [10]:
pdf_path = "./Docs/COMURS_GB_v4.pdf"

loader = PyPDFLoader(pdf_path)

docs = loader.load()

docs


[Document(metadata={'producer': 'Microsoft® Word 2019', 'creator': 'Microsoft® Word 2019', 'creationdate': '2025-12-10T22:41:52+05:30', 'author': 'SYMPO', 'moddate': '2025-12-10T22:41:52+05:30', 'source': './Docs/COMURS_GB_v4.pdf', 'total_pages': 5, 'page': 0, 'page_label': '1'}, page_content="1 \n \n \nEnhancing Natural Language Understanding in \nCompact LLMs via Task-Specific Knowledge \nDistillation  \n \nAbstract \n \nLarge language models (LLMs) have demonstrated outstanding \nperformance across a variety of natural language processing applications.  \nNatural Language Understanding (NLU) is a key characteristic that allows \nLLMs to interpret language, reasoning around context, and provide correct \nand coherent replies.  Although proprietary LLMs exhibit strong NLU \nbehavior, their computing requirements hinder practical application.  \nCompact models such as LLaMA 3.1: 8B and Mistral 7B provide a more \nefficient substitute, however these models  frequently fail to preserve \

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

chunks = splitter.split_documents(docs)

chunks

[Document(metadata={'producer': 'Microsoft® Word 2019', 'creator': 'Microsoft® Word 2019', 'creationdate': '2025-12-10T22:41:52+05:30', 'author': 'SYMPO', 'moddate': '2025-12-10T22:41:52+05:30', 'source': './Docs/COMURS_GB_v4.pdf', 'total_pages': 5, 'page': 0, 'page_label': '1'}, page_content='1 \n \n \nEnhancing Natural Language Understanding in \nCompact LLMs via Task-Specific Knowledge \nDistillation  \n \nAbstract \n \nLarge language models (LLMs) have demonstrated outstanding \nperformance across a variety of natural language processing applications.  \nNatural Language Understanding (NLU) is a key characteristic that allows \nLLMs to interpret language, reasoning around context, and provide correct \nand coherent replies.  Although proprietary LLMs exhibit strong NLU \nbehavior, their computing requirements hinder practical application.  \nCompact models such as LLaMA 3.1: 8B and Mistral 7B provide a more \nefficient substitute, however these models  frequently fail to preserve \

In [13]:
len(chunks)

17

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_openai.embeddings import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = Chroma.from_documents(documents=chunks, embedding=embedding_model, persist_directory="./chroma_db/", collection_name="comurs")

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
vectorstore.similarity_search("What is NLU? Tell me in one line", k=3)

## Use vector database

In [ ]:
vectorstore_persist = Chroma(persist_directory="./chroma_db/", 
                             embedding_function=embedding_model, 
                             collection_name="comurs")

In [ ]:
vectorstore_persist.similarity_search("What is NLU? Tell me in one line", k=3)

### Store embeddings in the existing local vector store

In [ ]:
vectorstore = Chroma(persist_directory="./chroma_db/", 
                             embedding_function=embedding_model)


vectorstore.add_documents(chunks)